In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms   # 데이터셋 로딩, normaliation시 필요한 transforms

from torchvision.datasets import ImageFolder
from torchvision import transforms   # 이미지 전처리용
from torch.utils.data import Dataset, DataLoader


import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import json
import time
import os
import copy
import random

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
%cd /content/drive/MyDrive/Datarhythm/DataAfterSplit

#!unzip -qq "/content/drive/MyDrive/Colab Notebooks/trainset/DataAfterSplit-20221103T154740Z-001.zip"

/content/drive/MyDrive/Datarhythm/DataAfterSplit


In [ ]:
## Data Load
batch_size  = 64
random_seed = 666
random.seed(random_seed)
torch.manual_seed(random_seed)

In [ ]:
#밸런스한 모델을 맞춤(75:25)
root='/content/drive/MyDrive/Datarhythm//DataAfterSplit'
trainset = ImageFolder(root= root+'/train',                   # 다운로드 받은 폴더의 root 경로를 지정합니다.
                      transform=transforms.Compose([
                          transforms.Resize((256, 256)),
                          transforms.CenterCrop(224),
                          transforms.ToTensor(), 
                          transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                      ]))

validset = ImageFolder(root= root+'/val',                   # 다운로드 받은 폴더의 root 경로를 지정합니다.
                      transform=transforms.Compose([
                          transforms.Resize((256, 256)),
                          transforms.CenterCrop(224),
                          transforms.ToTensor(), 
                          transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                      ]))

train_loader = torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(validset,batch_size=64,shuffle=False, num_workers=0)

In [ ]:
images, labels = next(iter(train_loader))
images.shape, labels.shape

(torch.Size([64, 3, 224, 224]), torch.Size([64]))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.sequential = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1), 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), 

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1), 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.MaxPool2d(2, 2), 

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1), 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128), 
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128), 
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Linear(7*7*128, 20)
    
    def forward(self, x):
        x = self.sequential(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x        

In [ ]:
model = CNNModel() # Model 생성
model.to(device)   # device 에 로드 (cpu or cuda)


# (128,8,8) = ouput(채널, 가로, 세로) 

CNNModel(
  (sequential): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNorm2d(128, eps=1e-05, momentum=0.1, 

In [ ]:
from torchsummary import summary

summary(model, (3, 244, 244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 244, 244]             896
       BatchNorm2d-2         [-1, 32, 244, 244]              64
              ReLU-3         [-1, 32, 244, 244]               0
         MaxPool2d-4         [-1, 32, 122, 122]               0
            Conv2d-5         [-1, 64, 122, 122]          18,496
       BatchNorm2d-6         [-1, 64, 122, 122]             128
              ReLU-7         [-1, 64, 122, 122]               0
           Dropout-8         [-1, 64, 122, 122]               0
         MaxPool2d-9           [-1, 64, 61, 61]               0
           Conv2d-10           [-1, 64, 61, 61]          36,928
      BatchNorm2d-11           [-1, 64, 61, 61]             128
             ReLU-12           [-1, 64, 61, 61]               0
        MaxPool2d-13           [-1, 64, 30, 30]               0
           Conv2d-14          [-1, 128,

In [ ]:
# 옵티마이저를 정의합니다. 옵티마이저에는 model.parameters()를 지정해야 합니다.
optimizer = optim.Adam(model.parameters(), lr=0.00054)

# 손실함수(loss function)을 지정합니다. Multi-Class Classification 이기 때문에 CrossEntropy 손실을 지정하였습니다.
loss_fn = nn.CrossEntropyLoss()

In [ ]:
from tqdm import tqdm  # Progress Bar 출력


In [ ]:
def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()
    
    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    
    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader)
    
    # mini-batch 학습을 시작합니다.
    for img, lbl in prograss_bar:
        # image, label 데이터를 device에 올립니다.
        img, lbl = img.to(device), lbl.to(device)
        
        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()
        
        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(img)
        
        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, lbl)
        
        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()
        
        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()
        
        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        _, pred = output.max(dim=1)
        
        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += pred.eq(lbl).sum().item()
        
        # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * img.size(0)
        
    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)
    
    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [ ]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다. 
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()
    
    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0
        
        # 배치별 evaluation을 진행합니다.
        for img, lbl in data_loader:
            # device에 데이터를 올립니다.
            img, lbl = img.to(device), lbl.to(device)
            
            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(img)
            
            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, pred = output.max(dim=1)
            
            # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(pred.eq(lbl)).item()
            
            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, lbl).item() * img.size(0)
        
        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)
        
        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [ ]:
# 최대 Epoch을 지정합니다.
num_epochs = 15

min_loss = np.inf

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(model, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(model, test_loader, loss_fn, device)   
    
    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[알림] val_loss가 {min_loss:.5f}에서 {val_loss:.5f}까지 개선되었다. 모델 저장!')
        min_loss = val_loss
        torch.save(model.state_dict(), 'Datarhythm_R.pth')
    
    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

100%|██████████| 188/188 [41:08<00:00, 13.13s/it]


[알림] val_loss가 inf에서 2.36519까지 개선되었다. 모델 저장!
epoch 01, loss: 2.53286, acc: 0.24417, val_loss: 2.36519, val_accuracy: 0.28700


100%|██████████| 188/188 [07:22<00:00,  2.35s/it]


[알림] val_loss가 2.36519에서 2.18442까지 개선되었다. 모델 저장!
epoch 02, loss: 1.99009, acc: 0.38917, val_loss: 2.18442, val_accuracy: 0.31100


100%|██████████| 188/188 [07:21<00:00,  2.35s/it]


...

epoch 13, loss: 0.32475, acc: 0.90125, val_loss: 1.43703, val_accuracy: 0.56700


100%|██████████| 188/188 [07:19<00:00,  2.34s/it]


epoch 14, loss: 0.26192, acc: 0.92892, val_loss: 1.39104, val_accuracy: 0.56650


100%|██████████| 188/188 [07:19<00:00,  2.34s/it]


epoch 15, loss: 0.21097, acc: 0.94358, val_loss: 1.35675, val_accuracy: 0.59200
